<a href="https://colab.research.google.com/github/waishun78/hungry-rag/blob/main/hungry_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Installing the necessary packages
!pip install --upgrade pip
!pip install 'farm-haystack[all-gpu]' ## or 'all-gpu' for the GPU-enabled dependencies

!pip install -U accelerate
!pip install bitsandbytes
!pip install SentencePiece
!pip install evaluate
!pip install bert_score
!pip install transformers

!pip install googlemaps

!pip install bert_score
# !apt install libgraphviz-dev
# !pip install pygraphviz

  Obtaining dependency information for pip from https://files.pythonhosted.org/packages/47/6a/453160888fab7c6a432a6e25f8afe6256d0d9f2cbd25971021da6491d899/pip-23.3.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 37.6 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 23.2.1
    Uninstalling pip-23.2.1:
      Successfully uninstalled pip-23.2.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 88.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.5/121.5 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 47.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 56.6 MB/s eta 0:00:00:00:01

In [ ]:
!nvidia-smi

In [5]:
# Import necessary modules
import os
import torch
import torch.nn as nn
import bitsandbytes as bnb
import accelerate
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
from transformers import LlamaTokenizer
from haystack.nodes.base import BaseComponent

2023-11-29 15:01:27,115	INFO util.py:129 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [6]:
load_in_8bit = True
model_name_l = "lmsys/vicuna-7b-v1.5"

model_l = AutoModelForCausalLM.from_pretrained(
    model_name_l,
    torch_dtype=torch.float16,
    load_in_8bit=load_in_8bit,
    device_map="auto"
)
tokenizer_l = LlamaTokenizer.from_pretrained(model_name_l)

/opt/conda/lib/python3.10/site-packages/ipywidgets/widgets/widget.py:503: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


### Testing Citation Generator

In [7]:
indian_result = {
    "generated_answer":"""1. A1 Indian Curry is the best choice as it has North Indian cuisine (which is required) and is open till 7:15 PM, which is a requirement as well. However, the reviews were mixed and one of them mentioned that the quality of one of the curries tasted off, so it is a bit risky to try.
    2. Tayyiba Briyani Restaurant @ Nexus One North is a good second choice as it has 4.7 rating and the reviews suggest that it has the best chicken biryani in Singapore. It is also open till 8 PM.\n
    3. Pangat North Indian Restaurant is the last choice as it has 3 rating and the reviews are not available. However, it is open till 7 PM, which is also a requirement.\n\nRank: 1 (A1 Indian Curry) > 2""",
    "context":[
    {
        'Option': 1,
        'Name': 'A1 Indian Curry North Indian & Lebanese Cuisine',
        'Address': '12/13 Pahang St, Singapore 198613',
        'Phone': '9423 8377',
        'Delivery Available': 'No',
        'Dine-In Available': 'Yes',
        'Opening Hours': 'Monday: 8:00\u202fAM\u2009–\u200912:15\u202fAM\nTuesday: 8:00\u202fAM\u2009–\u200912:15\u202fAM\nWednesday: 8:00\u202fAM\u2009–\u200912:15\u202fAM\nThursday: 8:00\u202fAM\u2009–\u200911:30\u202fPM\nFriday: 8:00\u202fAM\u2009–\u200912:30\u202fAM\nSaturday: 8:00\u202fAM\u2009–\u200912:30\u202fAM\nSunday: 8:00\u202fAM\u2009–\u200912:15\u202fAM',
        'Price Level': None,
        'Rating': '3.9',
        'Reviews': ['Service was ok...... we enjoyed the Fish and Chicken sets but didn\'t like 1 of the curries included as it tasted a little off...like it was left in the open for too long ...."basi...stale" and we have tasted many curries in the past and non-tasted ever like that.....\n\nDidn\'t wait too long for our food...... the gentleman who took our order......he made sure we understood that the food we ordered will not be the same as in the pics in the menu .... and it sure wasn\'t anything like the pics .....\n\nOverall we enjoyed our dinner.....just minus that dish....',
        'Indian food. We ordered chicken dum biryani,  it was very disappointing. Chicken portion extremely small and dry.\nI also ordered some falafel which was worse than the biryani.\nThe 1st photo below is what was advertised.. the 2nd photo is what was presented!!',
        'waited for 30mins for food and still not served yet until i asked, and they suddenly served us food that was cold and tasteless. will not even come back or recommend this place.']
    },
    {
        'Option': 2,
        'Name': 'Tayyiba Briyani Restaurant @ Nexus One North',
        'Address': '1 Fusionopolis Link, #01-03 Nexus @one-north - Space for Rent, Singapore 138542',
        'Phone': '6993 9648',
        'Delivery Available': 'Yes',
        'Dine-In Available': 'Yes',
        'Opening Hours': 'Monday: 10:30\u202fAM\u2009–\u20098:00\u202fPM\nTuesday: 10:30\u202fAM\u2009–\u20098:00\u202fPM\nWednesday: 10:30\u202fAM\u2009–\u20098:00\u202fPM\nThursday: 10:30\u202fAM\u2009–\u20098:00\u202fPM\nFriday: 10:30\u202fAM\u2009–\u20098:00\u202fPM\nSaturday: 11:30\u202fAM\u2009–\u20095:30\u202fPM\nSunday: Closed',
        'Price Level': None,
        'Rating': '4.7',
        'Reviews': ['Had the best chicken biryani in Singapore here. Awesome side dishes and raita. Highly recommended',
        'Excellent service... and yummilicious briyani.\nMust try!\nAmbience can certainly be enhanced to create an overall awesome dining experience.',
        'Amazing biryani paired with tender chicken that will literally fall off the bone! Whenever I have a biryani craving, I will always think of this restaurant. Owners are warm and friendly, providing great service. Highly recommended for anyone working around metropolis/fusionopolis to drop by!']
    },
    {
        'Option': 3,
        'Name': 'Pangat North Indian Restaurant',
        'Address': '1 Fusionopolis Way, Koufu, B2 #02 (PANGAT - MR5, 138632',
        'Phone': '8811 1141',
        'Delivery Available': 'Yes',
        'Dine-In Available': 'Yes',
        'Opening Hours': 'Monday: 10:30\u202fAM\u2009–\u20097:00\u202fPM\nTuesday: 10:30\u202fAM\u2009–\u20097:00\u202fPM\nWednesday: 10:30\u202fAM\u2009–\u20097:00\u202fPM\nThursday: 10:30\u202fAM\u2009–\u20097:00\u202fPM\nFriday: 10:30\u202fAM\u2009–\u20097:00\u202fPM\nSaturday: Closed\nSunday: Closed',
        'Price Level': None,
        'Rating': '3',
        'Reviews': ['', '']
    }
    ],
    "irrelevant_context":[
            {
        "Option": 1,
        "Name": "Waku Ghin",
        "Address": "Level 2 Dining, L2-03 The Shoppes at, 2 Bayfront Ave, Marina Bay Sands, 018956",
        "Price Level": "4 / 4 (0- Free, 1- Inexpensive, 2- Moderate, 3- Expensive, 4- Very Expensive, None- Price Data Not Available)",
        "Rating": 4.1,
        "Reviews":[
        "Absolutely amazing food; had the Omakase at the chef’s table. The umami and creamy sea urchin balanced with the sweetness of the shrimp and slight saltiness of the caviar to produce a full flavour profile with each mouthful. Loved the perfectly (teppanyaki) seared abalone with risoni and the A5 Kobe wagyu dipped in the egg yolk (it was so silky, tender and flavourful from the marbling). Service was excellent as well; they were jovial and attentive and made the experience delightful.",
        "This is a late review. I came here during valentines weekend. Both my friend and I had food poisoning and I was bed ridden for three days. Only decided to review now because my friend told me that her group of friends also got food poisoning from the same group of restaurants. So totally not worth it even tho the food was great. Spent 750 per pax to get food poisoning.. really ridiculous..",
        "We had some Izakaya food and beer before going for show. Food must be good considering the price is not cheap for beer food, but its a tad a bit too salty for me. Service is great of course. My favorite will be the lobster spaghetti, a bit of fusion."
        ]
    },
    {
        "Option": 2,
        "Name": "Sen of Japan",
        "Address": "2 Bayfront Ave, #01-86, Singapore 018972",
        "Price Level": "3 / 4 (0- Free, 1- Inexpensive, 2- Moderate, 3- Expensive, 4- Very Expensive, None- Price Data Not Available)",
        "Rating": 4.4,
        "Reviews":[
        "Lunch with a view. The marugo pizza is really nice and thin. Pork jowl is thinly slice. The thinnest I’ve seen. Nice in paring with the sauce. Black Cod fish is soft and nicely cook too. V worth it if you have American Express card, to enjoy 50% disc on the bill. This meal of 3 fish cause only $80+.",
        "Five stars for everything (except maybe the receptionist that received us.. kind of rude). The chefs were super friendly (sat at the counter), and I had nice small talk with them while eating.",
        "Excellent upscale Japanese restaurant great for a date. Lobster uni maki was out of this world. Amazing flavours and textures in a single mouthful."
        ]
    },
    {
        "Option": 3,
        "Name": "KOMA Singapore",
        "Address": "2 Bayfront Ave, #B1-67, Singapore 018972",
        "Price Level": "None / 4 (0- Free, 1- Inexpensive, 2- Moderate, 3- Expensive, 4- Very Expensive, None- Price Data Not Available)",
        "Rating": 4.3,
        "Reviews":[
        "Really nice restaurant with great food and perfect service. The atmosphere can get quite uncomfortable because it is always packed. Prices are reasonable for quality and location. Very recommended for date nights.",
        "Obviously, the main attraction would be the interior lighting. Can't deny the entire place makes a great backdrop for IG-worthy photos.",
        "Celebrated my wife birthday there recently, was a very nice place. Food was truly amazing and the staffs were very friendly and helpful in many ways."
        ]
    }
    ]
    
}

In [8]:
korean_result = {
    "generated_answer":"""1. Based on the given context and requirements, JINJJA Chicken @ Clementi Mall is the top choice with a price level of 2. It is located in the same mall as the user, has spicy Korean fried chicken, and bubble tea options.
        2. Jinjja Chicken Diner @ Bugis Plus is the second choice, with a price level of 2, it has located in the mall near the user's work and has spicy Korean fried chicken.
        3. Chir Chir Chir Fusion Chicken Factory Somerset is the third option, it is located in the same mall as the user and has spicy Korean fried chicken.</s>""",
    "context":[
    {
       "Option":1,
       "Name":"JINJJA Chicken @ Clementi Mall",
       "Address":"3155 Commonwealth Ave W, B1-28/29 The Clementi Mall, Singapore 129588",
       "Phone":"6262 2550",
       "Delivery Available":"Yes",
       "Dine-In Available":"Yes",
       "Opening Hours":"Monday: 11:00\u202fAM\u2009–\u20099:00\u202fPM\nTuesday: 11:00\u202fAM\u2009–\u20099:00\u202fPM\nWednesday: 11:00\u202fAM\u2009–\u20099:00\u202fPM\nThursday: 11:00\u202fAM\u2009–\u20099:00\u202fPM\nFriday: 11:00\u202fAM\u2009–\u20099:00\u202fPM\nSaturday: 11:00\u202fAM\u2009–\u20099:00\u202fPM\nSunday: 11:00\u202fAM\u2009–\u20099:00\u202fPM",
       "Price Level":2,
       "Rating":"4.1",
       "Reviews":[
          "Ordered the soy sauce garlic chicken drum sticks, the seaweed fries and the yuzu drink. The chicken has a rich soy sauce taste, is pretty tender inside, has crispy skin and is not spicy at all. I believe anything with one chilli beside it is not spicy at all. Would recommend the chicken and the fries.\n\nOne thing to note is that the chicken drum sticks are relatively big. I ordered 6 for 2 people thinking that it was small but it went from a snack to a full meal.",
          "Place is nice and clean. service is really minor, but this is fast food style. But a bit hard for new customer. But food. Really good. Very worth the dollar.",
          "Got the drumsticks mix - preferred the yangnyeom sauce to the soy as it was much tastier. The chicken was lightly seasoned, and the batter was crispy, but I have had more tender meat elsewhere.\n\nThe fries were underwhelming, so I do not recommend getting it as sides."
       ]
    },
    {
       "Option":2,
       "Name":"Chir Chir Fusion Chicken Factory Somerset",
       "Address":"313 Orchard Rd, B3 - 04 05 / 06, Singapore 238895",
       "Phone":"6509 8364",
       "Delivery Available":"Yes",
       "Dine-In Available":"Yes",
       "Opening Hours":"Monday: 11:00\u202fAM\u2009–\u20099:30\u202fPM\nTuesday: 11:00\u202fAM\u2009–\u20099:30\u202fPM\nWednesday: 11:00\u202fAM\u2009–\u20099:30\u202fPM\nThursday: 11:00\u202fAM\u2009–\u20099:30\u202fPM\nFriday: 11:00\u202fAM\u2009–\u200910:00\u202fPM\nSaturday: 11:00\u202fAM\u2009–\u200910:00\u202fPM\nSunday: 11:00\u202fAM\u2009–\u20099:30\u202fPM",
       "Price Level":"None",
       "Rating":"3.8",
       "Reviews":[
          "Visit Date 20 Nov 23\n\nChir Chir is a mandatory lunch spot when kids are visiting Singapore. When they were in school it used to be more often but now it is like once a year affair.\n\nVisited the 313 outlet after 1.5 years. Not sure exactly but couldn't locate the Bugis outlet in google. Perhaps it is closed down.\n\nToday it was not that satisfying experience I had expected, visiting after so long. We were seated quickly and the food came fast though.\n\nThe dishes we had\n- Garlicky Soy chicken. There was the familiar sweetness and soy although it tasted a little less garlicky than it used to be! However  with that, the chewy toppoki and crunchy sweet potato was great.\nLast time what made this chicken dish heavenly was a small bowl of spicy sauce on the side. This time they said, it was not for sale separately. Fair enough but the experience was not the same without it\n- Chir Cheese Fries Tower. The platter was beautifully presented with the inviting melted cheese on the side\n- Ice lemon tea was very good\n\nChir chir has interesting drinks. Amongst others ...\nPassion Yellow Tok! Tok!  A pale yellow Jug of mocktail. Today it was not available.\nBlue lemonade: By the time the blue jug with greenish and yellow hue came on the adjacent table we had ordered our ice lemon tea. but it was very tempting\n\nIt was a lot of food already but i still missed the Topokki and Cheese. It is generally delicious\n\nGuess once a year is good enough",
          "Google shown the closing hour is 9.30pm. When we reached at 8pm we were told that the restaurant is closing at 9pm. If there is a change in the operating hour, it should be updated. We made our way to Somerset only to find out that we have only less than an hour to dine. We felt that staff is rushing us to quickly finish our meal. Overall, we didnt enjoy the whole experience. Chir Chir used to be one of my go to place for korean fried chicken fix but after today experience, i will not want to go back. A few of the dishes that we would like to order was unavailable. Not sure if it is because we reached at 8pm or it is always unavailable. The honey butter chicken is the worst dish ever!!! I DO NOT RECOMMEND The Honey Butter Chicken. It used to be quite tasty with real honey and butter now it is replaced with some honey flavoured powder which smell and taste AWFUL. The spicy toppoki is not bad and the garlicky chicken with the cheese dip is not bad. Price wise its too pricey given that the quality of food is below average.\nOverall experience, sucks! Definitely not going back.",
          "Crispy chicken with mustard and salsa - 4 stars\nGood flavour, not sweet. Juicy with a crisp batter. The salsa that came with it was my favourite, but they gave a tiny portion. The server (with gelled back hair) walked past with a gigantic tub of salsa so I asked if I could have more. He said it only came with one portion. For $35 for the plate of tenders this seems rather ridiculous. I offered to pay for it and he still refused. Seems like really poor company policy if it really is. Docked stars for the poor service."
       ]
    },
    {
       "Option":3,
       "Name":"JINJJA Chicken Diner @ Bugis Plus",
       "Address":"201 Victoria St, #01-11 Bugis+, Singapore 188067",
       "Phone":"None",
       "Delivery Available":"Yes",
       "Dine-In Available":"Yes",
       "Opening Hours":"Monday: 11:00\u202fAM\u2009–\u20098:30\u202fPM\nTuesday: 11:00\u202fAM\u2009–\u20098:30\u202fPM\nWednesday: 11:00\u202fAM\u2009–\u20098:30\u202fPM\nThursday: 11:00\u202fAM\u2009–\u20098:30\u202fPM\nFriday: 11:00\u202fAM\u2009–\u20099:00\u202fPM\nSaturday: 11:00\u202fAM\u2009–\u20099:00\u202fPM\nSunday: 11:00\u202fAM\u2009–\u20098:30\u202fPM",
       "Price Level":2,
       "Rating":"4.2",
       "Reviews":[
          "Located on the first floor of Bugis Plus, they serve Korean-style fried chicken with flavorful sauce. Unfortunately, the chicken wasn't warm enough. The rice was incredibly soft! The staff provided good service, but the cleaner frequently attempted to clean the table even when we hadn't finished our food, which was quite annoying. The salad was good and came in a generous portion. Sometimes, they offer promotions for the chicken. Overall, it was a good experience.",
          "The food taste awefully cold and hard.  The fries, chicken omg taste so cold and hard.  My kids rejected them and we have to eat the food worth $50 ourselves!!   The rice is so cold too not even warm.  The 3 coke we had all almost no gas, it must been prepared way too earlier.  The skin suppose to be crispy but its very hard and cold.  Its the worst jinjja experience",
          "A big thumbs up for this restaurant����\nI was surprised that it literally had the best first impressions.\nEvery menu is getting better every time I taste. Not that much has shown the authenticity of Korean food. But still the best food to try out when you’re in Singapore."
       ]
    }
    ],
    "irrelevant_context":[
            {
        "Option": 1,
        "Name": "Waku Ghin",
        "Address": "Level 2 Dining, L2-03 The Shoppes at, 2 Bayfront Ave, Marina Bay Sands, 018956",
        "Price Level": "4 / 4 (0- Free, 1- Inexpensive, 2- Moderate, 3- Expensive, 4- Very Expensive, None- Price Data Not Available)",
        "Rating": 4.1,
        "Reviews":[
        "Absolutely amazing food; had the Omakase at the chef’s table. The umami and creamy sea urchin balanced with the sweetness of the shrimp and slight saltiness of the caviar to produce a full flavour profile with each mouthful. Loved the perfectly (teppanyaki) seared abalone with risoni and the A5 Kobe wagyu dipped in the egg yolk (it was so silky, tender and flavourful from the marbling). Service was excellent as well; they were jovial and attentive and made the experience delightful.",
        "This is a late review. I came here during valentines weekend. Both my friend and I had food poisoning and I was bed ridden for three days. Only decided to review now because my friend told me that her group of friends also got food poisoning from the same group of restaurants. So totally not worth it even tho the food was great. Spent 750 per pax to get food poisoning.. really ridiculous..",
        "We had some Izakaya food and beer before going for show. Food must be good considering the price is not cheap for beer food, but its a tad a bit too salty for me. Service is great of course. My favorite will be the lobster spaghetti, a bit of fusion."
        ]
    },
    {
        "Option": 2,
        "Name": "Sen of Japan",
        "Address": "2 Bayfront Ave, #01-86, Singapore 018972",
        "Price Level": "3 / 4 (0- Free, 1- Inexpensive, 2- Moderate, 3- Expensive, 4- Very Expensive, None- Price Data Not Available)",
        "Rating": 4.4,
        "Reviews":[
        "Lunch with a view. The marugo pizza is really nice and thin. Pork jowl is thinly slice. The thinnest I’ve seen. Nice in paring with the sauce. Black Cod fish is soft and nicely cook too. V worth it if you have American Express card, to enjoy 50% disc on the bill. This meal of 3 fish cause only $80+.",
        "Five stars for everything (except maybe the receptionist that received us.. kind of rude). The chefs were super friendly (sat at the counter), and I had nice small talk with them while eating.",
        "Excellent upscale Japanese restaurant great for a date. Lobster uni maki was out of this world. Amazing flavours and textures in a single mouthful."
        ]
    },
    {
        "Option": 3,
        "Name": "KOMA Singapore",
        "Address": "2 Bayfront Ave, #B1-67, Singapore 018972",
        "Price Level": "None / 4 (0- Free, 1- Inexpensive, 2- Moderate, 3- Expensive, 4- Very Expensive, None- Price Data Not Available)",
        "Rating": 4.3,
        "Reviews":[
        "Really nice restaurant with great food and perfect service. The atmosphere can get quite uncomfortable because it is always packed. Prices are reasonable for quality and location. Very recommended for date nights.",
        "Obviously, the main attraction would be the interior lighting. Can't deny the entire place makes a great backdrop for IG-worthy photos.",
        "Celebrated my wife birthday there recently, was a very nice place. Food was truly amazing and the staffs were very friendly and helpful in many ways."
        ]
    }
    ]
    
}

In [9]:
italian_result = {
    "generated_answer":"""1. Cicheti is the best choice as it has a great atmosphere and it is family-friendly. It also has great reviews for Italian food and has a variety of dishes at an affordable price. It is located in Little Italy and has lunch and dinner service.
            2. Positano Risto is a great option as it has good ratings and reviews, it is family-friendly and offers Italian cuisine. It is located in Bussorah</s>""",
    "context":[
   {
      "Option":1,
      "Name":"Little Italy - Katong",
      "Address":"297 Tanjong Katong Rd, Singapore 437080",
      "Phone":"8733 9903",
      "Delivery Available":"Yes",
      "Dine-In Available":"Yes",
      "Opening Hours":"Monday: Closed\nTuesday: 12:00\u2009–\u20092:45\u202fPM, 5:00\u2009–\u200910:30\u202fPM\nWednesday: 12:00\u2009–\u20092:45\u202fPM, 5:00\u2009–\u200910:30\u202fPM\nThursday: 12:00\u2009–\u20092:45\u202fPM, 5:00\u2009–\u200910:30\u202fPM\nFriday: 12:00\u2009–\u20092:45\u202fPM, 5:00\u2009–\u200910:30\u202fPM\nSaturday: 12:00\u2009–\u20092:45\u202fPM, 5:00\u2009–\u200910:30\u202fPM\nSunday: 12:00\u2009–\u20092:45\u202fPM, 5:00\u2009–\u200910:30\u202fPM",
      "Price Level":2,
      "Rating":"4.3",
      "Reviews":[
         "An impromptu visit turned into an unexpectedly fine meal.\n\nThe burrata was delicious, served with fresh ingredients including sweet tomatoes. The complementary bread blew my mind as it was freshly baked, fluffy and tasty! Went really well with the burrata. I could've just have the bread and burrata as a meal alone!\nFour-cheese pizza was good. Nicely baked thin crust.\nStaff was friendly. Interior/decorations were bluish simple in the good way.\n\nAn enjoyable Sunday brunch it was!",
         "Very nice and attentive service.\nAntipasti: Parma e Burrata and Gamberoni in Padella is superb nice!\nPasta: Spaghetti Alle Vongole taste is recommended\nDessert: Lava Cake is ok but not the chocolate flowy type i expected...\nDefinitely recommeded and will again...\nCant help it... have to order the Pizza takeaway for supper.....",
         "Visited the tanjong katong outlet a few months back. Waiters were hospitable. Food was surprisingly good, too, considering the fact that it was lunchtime and the restaurant was empty.\n\nI got the bufalina pizza and a lasagne. Pizza wasn't the traditional neapolitan style, but instead, it was thin and really crispy unlike most other Italian spots. Undercarriage char marks were a good indication of a solid high-temp oven. Pomodoro was fresh and tangy, coupled with proper fresh whole-milk mozzarella. You could also taste the same quality pomodoro in the lasagne, albeit my only complaint was that it felt a little dense.\n\nOverall, good portions and nice ambience. P.s. Do try their pizzas if you're a fan of thin crust."
      ]
   },
   {
      "Option":2,
      "Name":"Positano Risto",
      "Address":"66 Bussorah St, Singapore 199479",
      "Phone":"6292 1866",
      "Delivery Available":"Yes",
      "Dine-In Available":"Yes",
      "Opening Hours":"Monday: 12:00\u2009–\u200910:00\u202fPM\nTuesday: 12:00\u2009–\u200910:00\u202fPM\nWednesday: 12:00\u2009–\u200910:00\u202fPM\nThursday: 12:00\u2009–\u200910:00\u202fPM\nFriday: 12:00\u2009–\u200910:00\u202fPM\nSaturday: 12:00\u2009–\u200910:00\u202fPM\nSunday: 12:00\u2009–\u200910:00\u202fPM",
      "Price Level":2,
      "Rating":"4.3",
      "Reviews":[
         "Service is good with friendly staff. Food is served quite fast, they are tasty, but all tend to be on the salty side.\n\nThe call out for us is the squid ink seafood spaghetti and the five cheeses pizza. The thin crust was impressive. The spaghetti cooked Al Dante.\n\nThe lobster bisque was nice but too salty; the soft shell crab for the  linguine was not fresh, fried in thick batter and tasted weird.\n\nThe ambience is at best so so…",
         "Went there for a gathering and the acoustics was not made for large gatherings. I do like the salad though the waiter did not ask if I wanted olives in my salad. Service was prompt for a bustling place. The ambience around the restaurant is nice as kg glam has a nice vibe. Nice area to walk around after the meal.",
         "Great place for western halal food. We got a few dishes for sharing as we were spoilt for choice.\n\nService was personable and staff was upfront that they had sold out of some items, that we should order dessert in advance and that we would have to wait, which I appreciated. They were also very attentive  but also let us sit and not chase us away when we finished our meal.\n\nFood was delicious and I’ll be back to try more dishes for sure."
      ]
   },
   {
      "Option":3,
      "Name":"Cicheti",
      "Address":"52 Kandahar St, Singapore 198901",
      "Phone":"9725 6255",
      "Delivery Available":"Yes",
      "Dine-In Available":"Yes",
      "Opening Hours":"Monday: 12:00\u2009–\u20092:30\u202fPM, 6:00\u2009–\u20099:30\u202fPM\nTuesday: 12:00\u2009–\u20092:30\u202fPM, 6:00\u2009–\u20099:30\u202fPM\nWednesday: 12:00\u2009–\u20092:30\u202fPM, 6:00\u2009–\u20099:30\u202fPM\nThursday: 12:00\u2009–\u20092:30\u202fPM, 6:00\u2009–\u20099:30\u202fPM\nFriday: 12:00\u2009–\u20092:30\u202fPM, 6:00\u2009–\u200910:00\u202fPM\nSaturday: 12:00\u2009–\u20092:30\u202fPM, 6:00\u2009–\u200910:00\u202fPM\nSunday: 12:00\u2009–\u20092:30\u202fPM, 6:00\u2009–\u20099:30\u202fPM",
      "Price Level":3,
      "Rating":"4.3",
      "Reviews":[
         "was told that it was the best pizza in Singapore... it was pretty good, but not the best. the open concept kitchen is pretty cool, where you can watch the kitchen staff prep food. the interior was very noisy though, and the tables were quite cramped. you can't really fit two pizzas on a table for two with room for drinks.\n\nthe menu was predominantly very meaty, and not very many lighter vegetable options. prices were reasonable.\n\njust FYI the toilet has a slatted door that can be semi-seen through. soooo if you're highly private then ya ��",
         "The Pork Thermohawk is sweet, juicy and strong aroma of char.\nThe mussels with the fresh tomatoes.. sweet, sour.\nOnly advice do not seat near the door during the afternoon the Heat gets in and was sweating in addition to the guests coming in and out , quite horrid to be honest, other than that all good ��",
         "Came here for lunch one day as my friend recommended this place and was not disappointed.\n\nMust order would definitely be the pizzas- doughy with a good amount of chewiness but not too thick. Calamari was also fried to perfection without being too oily. Crab meat pasta was ok although I felt the flavours didn’t really infuse well into the pasta. The molten chocolate cake was also just so so I definitely have had better.\n\nOverall a good place for a special lunch/ dinner!"
      ]
   }
    ],
    "irrelevant_context":[
            {
        "Option": 1,
        "Name": "Waku Ghin",
        "Address": "Level 2 Dining, L2-03 The Shoppes at, 2 Bayfront Ave, Marina Bay Sands, 018956",
        "Price Level": "4 / 4 (0- Free, 1- Inexpensive, 2- Moderate, 3- Expensive, 4- Very Expensive, None- Price Data Not Available)",
        "Rating": 4.1,
        "Reviews":[
        "Absolutely amazing food; had the Omakase at the chef’s table. The umami and creamy sea urchin balanced with the sweetness of the shrimp and slight saltiness of the caviar to produce a full flavour profile with each mouthful. Loved the perfectly (teppanyaki) seared abalone with risoni and the A5 Kobe wagyu dipped in the egg yolk (it was so silky, tender and flavourful from the marbling). Service was excellent as well; they were jovial and attentive and made the experience delightful.",
        "This is a late review. I came here during valentines weekend. Both my friend and I had food poisoning and I was bed ridden for three days. Only decided to review now because my friend told me that her group of friends also got food poisoning from the same group of restaurants. So totally not worth it even tho the food was great. Spent 750 per pax to get food poisoning.. really ridiculous..",
        "We had some Izakaya food and beer before going for show. Food must be good considering the price is not cheap for beer food, but its a tad a bit too salty for me. Service is great of course. My favorite will be the lobster spaghetti, a bit of fusion."
        ]
    },
    {
        "Option": 2,
        "Name": "Sen of Japan",
        "Address": "2 Bayfront Ave, #01-86, Singapore 018972",
        "Price Level": "3 / 4 (0- Free, 1- Inexpensive, 2- Moderate, 3- Expensive, 4- Very Expensive, None- Price Data Not Available)",
        "Rating": 4.4,
        "Reviews":[
        "Lunch with a view. The marugo pizza is really nice and thin. Pork jowl is thinly slice. The thinnest I’ve seen. Nice in paring with the sauce. Black Cod fish is soft and nicely cook too. V worth it if you have American Express card, to enjoy 50% disc on the bill. This meal of 3 fish cause only $80+.",
        "Five stars for everything (except maybe the receptionist that received us.. kind of rude). The chefs were super friendly (sat at the counter), and I had nice small talk with them while eating.",
        "Excellent upscale Japanese restaurant great for a date. Lobster uni maki was out of this world. Amazing flavours and textures in a single mouthful."
        ]
    },
    {
        "Option": 3,
        "Name": "KOMA Singapore",
        "Address": "2 Bayfront Ave, #B1-67, Singapore 018972",
        "Price Level": "None / 4 (0- Free, 1- Inexpensive, 2- Moderate, 3- Expensive, 4- Very Expensive, None- Price Data Not Available)",
        "Rating": 4.3,
        "Reviews":[
        "Really nice restaurant with great food and perfect service. The atmosphere can get quite uncomfortable because it is always packed. Prices are reasonable for quality and location. Very recommended for date nights.",
        "Obviously, the main attraction would be the interior lighting. Can't deny the entire place makes a great backdrop for IG-worthy photos.",
        "Celebrated my wife birthday there recently, was a very nice place. Food was truly amazing and the staffs were very friendly and helpful in many ways."
        ]
    }
    ]
    
}

In [10]:
import json
from bert_score import score

class BERTScore_Citation_Generator(BaseComponent):
    outgoing_edges = 1

    def __init__(self, threshold):
        self.threshold = threshold

    def run(self, query: dict):
        max_F1 = 0
        chosen_index = -1
        annotation_ls = []
        for i in range(len(query["context"])):
            restaurant_stat = json.dumps(query["context"][i])
            P, R, F1 = score([restaurant_stat], [query["generated_answer"]], lang="en", verbose=True)
            print(f'P{P, R, F1} for {restaurant_stat[:50]}')
            
        for i in range(len(query["irrelevant_context"])):
            restaurant_stat = json.dumps(query["irrelevant_context"][i])
            P, R, F1 = score([restaurant_stat], [query["generated_answer"]], lang="en", verbose=True)
            print(f'P{P, R, F1} for {restaurant_stat[:50]}')

            
        return {"test":annotation_ls}, "output_1"

    def run_batch(self, queries):
        # Insert code here to manipulate the input and produce an output dictionary
        output = []
        for query in queries:
            output.append(self.run(query))
        return output, "output_1"

In [11]:
from haystack import Pipeline
bert_citation = BERTScore_Citation_Generator(0.7)

# Create a Haystack pipeline
p4 = Pipeline()
p4.add_node(component=bert_citation, name="BERTScore_Citation_Generator", inputs=["Query"])

In [12]:
result_test = p4.run(query = indian_result)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


/opt/conda/lib/python3.10/site-packages/ipywidgets/widgets/widget.py:503: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


...

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.07 seconds, 13.71 sentences/sec
P(tensor([0.8000]), tensor([0.8144]), tensor([0.8071])) for {"Option": 3, "Name": "KOMA Singapore", "Address":


In [13]:
result_test = p4.run(query = korean_result)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.08 seconds, 12.10 sentences/sec
P(tensor([0.7889]), tensor([0.8023]), tensor([0.7955])) for {"Option": 3, "Name": "KOMA Singapore", "Address":


In [14]:
result_test = p4.run(query = italian_result)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.07 seconds, 13.60 sentences/sec
P(tensor([0.7964]), tensor([0.8278]), tensor([0.8118])) for {"Option": 3, "Name": "KOMA Singapore", "Address":


In [15]:
import spacy

nlp = spacy.load("en_core_web_sm")
def extract_keywords(text):
    doc = nlp(text)
    return [token.text for token in doc if token.is_stop == False and token.is_punct == False]

class Matching_Citation_Generator(BaseComponent):
    outgoing_edges = 1

    def __init__(self, threshold = 0.4):
        self.threshold = threshold
        pass
    
    def run(self, query: dict):
        
        annotation_ls = []

        generated_answer_tokens = extract_keywords(query["generated_answer"])

        for i in range(len(query["context"])):
            print(f'Comparing context {i}:{query["context"][i]["Name"]}')
            if query["context"][i].get("Name") in query["generated_answer"]:
                print("Restaurant name in answer!")
            else:
                print("Restaurant name NOT in answer!")

            restaurant_stat = json.dumps(query["context"][i])

            restaurant_tokens = extract_keywords(restaurant_stat)

            common_keywords = set(generated_answer_tokens).intersection(set(restaurant_tokens))
            print("Common Keywords:", common_keywords, " Common:", len(common_keywords), "Generated Length:",len(generated_answer_tokens), " Percentage:", len(common_keywords)/len(generated_answer_tokens))
            
        for i in range(len(query["irrelevant_context"])):
            print(f'Comparing context {i}:{query["irrelevant_context"][i]["Name"]}')
            if query["irrelevant_context"][i].get("Name") in query["generated_answer"]:
                print("Restaurant name in answer!")
            else:
                print("Restaurant name NOT in answer!")

            restaurant_stat = json.dumps(query["irrelevant_context"][i])

            restaurant_tokens = extract_keywords(restaurant_stat)

            common_keywords = set(generated_answer_tokens).intersection(set(restaurant_tokens))
            print("Common Keywords:", common_keywords, " Common:", len(common_keywords), "Generated Length:",len(generated_answer_tokens), " Percentage:", len(common_keywords)/len(generated_answer_tokens))
        return {"test":"test"}, "output_1"

    def run_batch(self, queries):
        # Insert code here to manipulate the input and produce an output dictionary
        output = []
        for query in queries:
            output.append(self.run(query))
        return output, "output_1"

In [ ]:
matching_citation = Matching_Citation_Generator()

# Create a Haystack pipeline
p4 = Pipeline()
p4.add_node(component=matching_citation, name="Matching_Citation_Generator", inputs=["Query"])

In [87]:
result_test = p4.run(query = indian_result)

Comparing context 0:A1 Indian Curry North Indian & Lebanese Cuisine
Restaurant name NOT in answer!
Common Keywords: {'A1', 'Curry', 'Indian', 'curries', '1', 'tasted', 'biryani', 'North', 'chicken', 'Singapore', 'open'}  Common: 11 Generated Length: 70  Percentage: 0.15714285714285714
Comparing context 1:Tayyiba Briyani Restaurant @ Nexus One North
Restaurant name in answer!
Common Keywords: {'Nexus', '1', 'best', 'biryani', 'North', 'chicken', 'Singapore', 'Tayyiba', '2', 'Restaurant', '4.7', 'Briyani'}  Common: 12 Generated Length: 70  Percentage: 0.17142857142857143
Comparing context 2:Pangat North Indian Restaurant
Restaurant name in answer!
Common Keywords: {'Indian', 'Pangat', '1', '3', 'North', 'Restaurant'}  Common: 6 Generated Length: 70  Percentage: 0.08571428571428572
Comparing context 0:Waku Ghin
Restaurant name NOT in answer!
Common Keywords: {'good', '1', '2', 'bit'}  Common: 4 Generated Length: 70  Percentage: 0.05714285714285714
Comparing context 1:Sen of Japan
Restaura

In [88]:
result_test = p4.run(query = korean_result)

Comparing context 0:JINJJA Chicken @ Clementi Mall
Restaurant name in answer!
Common Keywords: {'1', 'spicy', 'Clementi', 'Chicken', 'chicken', 'JINJJA', 'Mall', '2'}  Common: 8 Generated Length: 62  Percentage: 0.12903225806451613
Comparing context 1:Chir Chir Fusion Chicken Factory Somerset
Restaurant name in answer!
Common Keywords: {'Chir', 'Bugis', 'Fusion', 'spicy', 'Chicken', 'chicken', 'tea', 'given', 'Factory', '2', 'Somerset', 'fried'}  Common: 12 Generated Length: 62  Percentage: 0.1935483870967742
Comparing context 2:JINJJA Chicken Diner @ Bugis Plus
Restaurant name NOT in answer!
Common Keywords: {'Bugis', 'Chicken', '3', 'Diner', 'chicken', 'JINJJA', 'Plus', 'fried', 'Korean', '2'}  Common: 10 Generated Length: 62  Percentage: 0.16129032258064516
Comparing context 0:Waku Ghin
Restaurant name NOT in answer!
Common Keywords: {'price', '1', '2'}  Common: 3 Generated Length: 62  Percentage: 0.04838709677419355
Comparing context 1:Sen of Japan
Restaurant name NOT in answer!
Co

In [89]:
result_test = p4.run(query = italian_result)

Comparing context 0:Little Italy - Katong
Restaurant name NOT in answer!
Common Keywords: {'good', 'Little', 'Italian', '1', 'Italy', 'friendly', '2'}  Common: 7 Generated Length: 39  Percentage: 0.1794871794871795
Comparing context 1:Positano Risto
Restaurant name in answer!
Common Keywords: {'good', 'food', 'best', 'dishes', 'friendly', 'Positano', 'Risto', '2'}  Common: 8 Generated Length: 39  Percentage: 0.20512820512820512
Comparing context 2:Cicheti
Restaurant name in answer!
Common Keywords: {'good', 'Cicheti', 'food', 'best', 'lunch', 'dinner'}  Common: 6 Generated Length: 39  Percentage: 0.15384615384615385
Comparing context 0:Waku Ghin
Restaurant name NOT in answer!
Common Keywords: {'good', 'food', '1', 'price', 'great', '2'}  Common: 6 Generated Length: 39  Percentage: 0.15384615384615385
Comparing context 1:Sen of Japan
Restaurant name NOT in answer!
Common Keywords: {'great', 'friendly', '2'}  Common: 3 Generated Length: 39  Percentage: 0.07692307692307693
Comparing conte

In [25]:
import spacy

def from_context_prompt_generator(context, answer):
    prompt = f"""
    In the following task, you are to determine whether the LLM's output is relevant to the provided context. Two examples are given to illustrate how to assess relevance. Carefully compare the LLM's output with the given context, and then state whether the output is supported by the context.
    EXAMPLE 1:
    LLM Output:
        1. Sen of Japan is the best choice as it serves sushi ('Lobster uni maki'), and it has the highest rating of 4.3 out of the other options. Additionally, it was described as 'great for a date' in a review and is open until 22:00.
        2. A close second is KOMA Singapore. It has excellent interior lighting for a romantic atmosphere and quality food. It also has a rating of 4.3 but doesn't explicitly mention serving sushi.
        3. Waku Ghin is the third choice as it is more expensive than the others. A review stated it was 'totally not worth it even though the food was great'.

    Context for Example 1: 
        Name: KOMA Singapore
        Address: 2 Bayfront Ave, # B1 - 67, Singapore 018972
        Price Level: None / 4
        Rating: 4.3
        Review 1: Really nice restaurant with great food and perfect service. The atmosphere can get quite uncomfortable because it is always packed. Prices are reasonable for quality and location. Very recommended for date nights.
        Review 2: Obviously, the main attraction would be the interior lighting. Can't deny the entire place makes a great backdrop for IG-worthy photos.
        Review 3: Celebrated my wife's birthday there recently, was a very nice place. Food was truly amazing and the staffs were very friendly and helpful in many ways.

    Is the LLM Output Supported by the Context for Example 1?
    Answer: Yes, it is relevant.

    EXAMPLE 2:
    LLM Output:
        1. Sen of Japan is the best choice as it serves sushi ('Lobster uni maki'), and it has the highest rating of 4.3 out of the other options. Additionally, it was described as 'great for a date' in a review and is open until 22:00.
        2. A close second is KOMA Singapore. It has excellent interior lighting for a romantic atmosphere and quality food. It also has a rating of 4.3 but doesn't explicitly mention serving sushi.
        3. Waku Ghin is the third choice as it is more expensive than the others. A review stated it was 'totally not worth it even though the food was great'.

    Context for Example 2: 
        Name: Saizerya
        Address: 3 Changi City Ave, Singapore 890233
        Price Level: 3 / 4
        Rating: 2.3
        Review 1: Terrible restaurant with bad service, food is bad.
        Review 2: Pasta was delicious and was really cheap. Enjoyed the food.
        Review 3: Had a medium time at Saizerya. It is what you expect.

    Is the LLM Output Supported by the Context for Example 2?
    Answer: No, it is not relevant.

    ACTUAL TASK:
    LLM Output:
    {context}

    Context:
    {answer}

    Is the LLM Output Supported by the Context in the Actual Task?
    Answer: 
    """
    return prompt

class LLM_Citation_Generator(BaseComponent):
    outgoing_edges = 1

    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
    
    def run(self, query: dict):
        
        for i in range(len(query["context"])):
            prompt = from_context_prompt_generator(query["context"][i], query["generated_answer"])
            input = self.tokenizer(prompt, return_tensors='pt')
            input_ids = input["input_ids"].to("cuda")
            temperature=0.8
            with torch.no_grad():
                generation_output = self.model.generate(
                    input_ids=input_ids,
                    temperature=temperature,
                    top_p = 1.0,
                    do_sample=True,
                    return_dict_in_generate=True,
                    max_new_tokens=200,
                  )
            s = generation_output.sequences[0][len(input_ids[0]):]
            generated_answer= self.tokenizer.decode(s)
            print(f'\n____LLM judgement:_________\n{generated_answer}, \n context:{query["context"][i]["Name"]}, \nis relevant:{"yes" in generated_answer.lower() or "is relevant" in generated_answer.lower()} \n\n')

            del s
            del input
            del input_ids
            torch.cuda.empty_cache()
        
        for i in range(len(query["irrelevant_context"])):
            prompt = from_context_prompt_generator(query["irrelevant_context"][i], query["generated_answer"])
            input = self.tokenizer(prompt, return_tensors='pt')
            input_ids = input["input_ids"].to("cuda")
            with torch.no_grad():
                generation_output = self.model.generate(
                    input_ids=input_ids,
                    temperature=temperature,
                    top_p = 1.0,
                    do_sample=True,
                    return_dict_in_generate=True,
                    max_new_tokens=200,
                  )
            s = generation_output.sequences[0][len(input_ids[0]):]
            generated_answer= self.tokenizer.decode(s)
            print(f'\n____LLM judgement:_________\n{generated_answer}, \n context:{query["irrelevant_context"][i]["Name"]},\nis relevant:{"yes" in generated_answer.lower()} \n\n')

            del s
            del input
            del input_ids
            torch.cuda.empty_cache()
            
        return {"test":"test"}, "output_1"

    def run_batch(self, queries):
        # Insert code here to manipulate the input and produce an output dictionary
        output = []
        for query in queries:
            output.append(self.run(query))
        return output, "output_1"

In [26]:
torch.cuda.empty_cache()

In [27]:
LLM_c = LLM_Citation_Generator(model_l, tokenizer_l)

# Create a Haystack pipeline
p4 = Pipeline()
p4.add_node(component=LLM_c, name="LLM_Citation_Generator", inputs=["Query"])

In [28]:
result_test = p4.run(query = indian_result)


____LLM judgement:_________


The LLM output is not supported by the given context in the actual task. The LLM output is based on the options provided in the output, which are not relevant to the requirements given in the task.</s>, 
 context:A1 Indian Curry North Indian & Lebanese Cuisine, 
is relevant:False 



____LLM judgement:_________

The given LLM output is not relevant to the provided context as it does not match the criteria of being open till 7:15 PM and having North Indian cuisine, which are the requirements mentioned in the context. It is also not the best choice as per the given ratings and reviews, it is ranked 2nd and one of the reviews mentioned that the quality of one of the curries tasted off.</s>, 
 context:Tayyiba Briyani Restaurant @ Nexus One North, 
is relevant:False 



____LLM judgement:_________

As the LLM output is not mentioned in the given context, it is not supported by the context.</s>, 
 context:Pangat North Indian Restaurant, 
is relevant:False 



_

In [29]:
result_test = p4.run(query = korean_result)


____LLM judgement:_________


The LLM's output is not relevant to the provided context. The LLM's output is for a different location, with different dining options, and a different price level compared to the given context.</s>, 
 context:JINJJA Chicken @ Clementi Mall, 
is relevant:False 



____LLM judgement:_________


번역결과  
The LLM's output is not relevant to the provided context. The LLM recommends a restaurant (Chir Chir Fusion Chicken Factory) that is not even listed in the options provided in the context. It also includes a review that is not positive about that specific restaurant. Therefore, the LLM's output is not supported by the given context.</s>, 
 context:Chir Chir Fusion Chicken Factory Somerset, 
is relevant:False 



____LLM judgement:_________

Supported by the context: Yes</s>, 
 context:JINJJA Chicken Diner @ Bugis Plus, 
is relevant:True 



____LLM judgement:_________
1. No, it is not relevant.</s>, 
 context:Waku Ghin,
is relevant:False 



____LLM judgement:

In [30]:
result_test = p4.run(query = italian_result)


____LLM judgement:_________


To determine if the LLM's output is relevant to the given context, compare the LLM's output with the context provided in the task.

The LLM's output only mentions a single restaurant, Little Italy - Katong, and provides some basic information about the restaurant. However, the context provided in the task requires the LLM to compare and recommend two Italian restaurants in Little</s>, 
 context:Little Italy - Katong, 
is relevant:True 



____LLM judgement:_________

    No, the LLM output is not relevant to the provided context. The context describes the best</s>, 
 context:Positano Risto, 
is relevant:False 



____LLM judgement:_________

Is the LLM's output relevant to the given context for the actual task?</s>, 
 context:Cicheti, 
is relevant:False 



____LLM judgement:_________


LLM Output:
{'Option': 3, 'Name': 'Waku Ghin', 'Address': 'Level 2 Dining, L2-03 The Shoppes at, 2 Bayfront Ave, Marina Bay Sands, 018956', 'Price Level': '4 / 4 (0- Free,